In [1]:
import sys
import os
from pathlib import Path
from dotenv import load_dotenv
import pandas as pd
import optuna
import importlib

sys.path.append(os.path.abspath(".."))

import src.models.xgb.xgb_optuna_optimizer as op
import src.models.xgb.xgb_cv_trainer as cv

In [2]:
# データの読み込み
env_path = Path.cwd().parent / ".env"
load_dotenv(dotenv_path=env_path)
url = os.environ.get("OPTUNA_STORAGE_URL")

tr_df1 = pd.read_csv("../artifacts/features/tr_df1.csv")
test_df1 = pd.read_csv("../artifacts/features/test_df1.csv")

cat_cols = ["Sex"]

In [3]:
# tr_df1
importlib.reload(cv)
importlib.reload(op)
objective = op.create_objective(
    tr_df1,
    early_stopping_rounds=500,
    tree_method="gpu_hist",
    n_jobs=10,
    cat_cols=cat_cols
)

random_sampler = optuna.samplers.RandomSampler(seed=42)

op.run_optuna_search(
    objective,
    n_trials=50,
    n_jobs=1,
    study_name="xgb_v1",
    storage=url,
    sampler=random_sampler
)

[I 2025-07-18 13:20:39,457] Using an existing study with name 'xgb_v1' instead of creating a new one.


  0%|          | 0/50 [00:00<?, ?it/s]

[0]	train-rmse:0.94458	eval-rmse:0.94374
[100]	train-rmse:0.15198	eval-rmse:0.15162
[200]	train-rmse:0.06787	eval-rmse:0.06754
[300]	train-rmse:0.06190	eval-rmse:0.06171
[400]	train-rmse:0.06074	eval-rmse:0.06071
[500]	train-rmse:0.06024	eval-rmse:0.06034
[600]	train-rmse:0.05993	eval-rmse:0.06015
[700]	train-rmse:0.05971	eval-rmse:0.06002
[800]	train-rmse:0.05955	eval-rmse:0.05994
[900]	train-rmse:0.05941	eval-rmse:0.05988
[1000]	train-rmse:0.05928	eval-rmse:0.05985
[1100]	train-rmse:0.05917	eval-rmse:0.05983
[1200]	train-rmse:0.05907	eval-rmse:0.05981
[1300]	train-rmse:0.05897	eval-rmse:0.05980
[1400]	train-rmse:0.05888	eval-rmse:0.05978
[1500]	train-rmse:0.05879	eval-rmse:0.05977
[1600]	train-rmse:0.05871	eval-rmse:0.05976
[1700]	train-rmse:0.05864	eval-rmse:0.05975
[1800]	train-rmse:0.05856	eval-rmse:0.05974
[1900]	train-rmse:0.05849	eval-rmse:0.05974
[2000]	train-rmse:0.05843	eval-rmse:0.05973
[2100]	train-rmse:0.05837	eval-rmse:0.05974
[2200]	train-rmse:0.05831	eval-rmse:0.05974


In [10]:
# ID19のfull train
params = {
    "max_depth": 8,
    "min_child_weight": 29.803558296988122,
    "colsample_bytree": 0.3970689908418626,
    "subsample": 0.856532598287343,
    "reg_alpha": 9.002010481036006,
    "reg_lambda": 0.004363285541408288,
    "learning_rate": 0.02
}

trainer = cv.XGBCVTrainer(params=params, cat_cols=cat_cols)
trainer.full_train(tr_df4, test_df4, 4613, 19)

Training time: 00:06:44
Successfully saved test predictions to ../artifacts/test_preds/full/test_full_19.npy


In [11]:
trainer = cv.XGBCVTrainer(params=params, cat_cols=cat_cols)
oof_preds, test_preds = trainer.fit(tr_df4, test_df4)


Fold 1
[0]	train-map@3:0.30832	eval-map@3:0.29452
[100]	train-map@3:0.35338	eval-map@3:0.32591
[200]	train-map@3:0.36443	eval-map@3:0.33122
[300]	train-map@3:0.37450	eval-map@3:0.33485
[400]	train-map@3:0.38393	eval-map@3:0.33801
[500]	train-map@3:0.39284	eval-map@3:0.34143
[600]	train-map@3:0.40138	eval-map@3:0.34408
[700]	train-map@3:0.40998	eval-map@3:0.34669
[800]	train-map@3:0.41757	eval-map@3:0.34941
[900]	train-map@3:0.42496	eval-map@3:0.35173
[1000]	train-map@3:0.43178	eval-map@3:0.35385
[1100]	train-map@3:0.43788	eval-map@3:0.35578
[1200]	train-map@3:0.44386	eval-map@3:0.35717
[1300]	train-map@3:0.44927	eval-map@3:0.35839
[1400]	train-map@3:0.45420	eval-map@3:0.35971
[1500]	train-map@3:0.45874	eval-map@3:0.36083
[1600]	train-map@3:0.46290	eval-map@3:0.36188
[1700]	train-map@3:0.46688	eval-map@3:0.36273
[1800]	train-map@3:0.47038	eval-map@3:0.36334
[1900]	train-map@3:0.47386	eval-map@3:0.36418
[2000]	train-map@3:0.47700	eval-map@3:0.36457
[2100]	train-map@3:0.48011	eval-map@3: